In [1]:
import os

In [2]:
%pwd

'c:\\Users\\user\\Documents\\end to end ml project\\end-to-end-ml-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\user\\Documents\\end to end ml project\\end-to-end-ml-project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig: 
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = config.source_url,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename} of size: {get_size(filename)}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file} of size: {get_size(self.config.local_data_file)}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path) 

In [11]:
try:
    config =   ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-20 18:12:53,042: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-20 18:12:53,044: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-20 18:12:53,046: INFO: common: created directory at: artifacts]
[2025-05-20 18:12:53,048: INFO: common: created directory at: artifacts/data_ingestion]


[2025-05-20 18:12:57,258: INFO: 3390841045: Downloaded file: artifacts\data_ingestion\data.zip of size: ~ 11345 KB]
